In [3]:
import xlrd
import re
import csv
import datetime
import os

default_data = {
    "blood": [7.0, 2.03, 4.389],
    "blood_test": [7.0, 29.0, 62.7],
    "thyroid": [2.82, 1.234, 0.912, 7.35, 2.29, 80.816, 39.46],
    "ana18": [0],
    "AQP4IgG": [0],
    "t_b": [236, 11.05, 215, 10.9, 1580, 77.91, 868, 622, 31.1, 42.2],
    "il_6": [2.0],
    "medicine": [0, 0, 0, 0, 0, 1],
    "visit": [],
    "edss": [0.0],
    "vitd": [18.6, 49.88],
    "ig3": [10.39, 1.83, 1.00],
    "wbc": [8.0]
}
f = xlrd.open_workbook('data/original_sheet/NMOSD-2016-2017-清洗隐私-补充.xls')
patients_1 = f.sheets()[0]
visit_1 = f.sheets()[1]
breakout_1 = f.sheets()[2]
edss_1 = f.sheets()[3]
blood_1 = f.sheets()[4]
thyroid_1 = f.sheets()[5]
ana18_1 = f.sheets()[6]
aqp4igg_1 = f.sheets()[7]
vitd_1 = f.sheets()[8]
t_b_1 = f.sheets()[9]
ig3_1 = f.sheets()[10]
il_6_1 = f.sheets()[11]
wbc_1 = f.sheets()[12]
csf_1 = f.sheets()[13]
ocb_1 = f.sheets()[14]
medicine_1 = f.sheets()[16] 
f = xlrd.open_workbook('data/original_sheet/NMOSD-2018-2021-清洗隐私-补充.xls')
patients_2 = f.sheets()[0]
visit_2 = f.sheets()[1]
breakout_2 = f.sheets()[2]
edss_2 = f.sheets()[3]
blood_2 = f.sheets()[4]
thyroid_2 = f.sheets()[5]
ana18_2 = f.sheets()[6]
aqp4igg_2 = f.sheets()[7]
vitd_2 = f.sheets()[8]
t_b_2 = f.sheets()[9]
ig3_2 = f.sheets()[10]
il_6_2 = f.sheets()[11]
wbc_2 = f.sheets()[12]
csf_2 = f.sheets()[13]
ocb_2 = f.sheets()[14]
medicine_2 = f.sheets()[16] 

f = xlrd.open_workbook('data/original_sheet/血常规数据-20221017.xls')
blood_new = f.sheets()[1]

In [4]:
#将字符串转换为float型数据
def str_to_float(s):
    if isinstance(s, float) or isinstance(s, int):
        return s
    else:
        k1 = re.compile(r'(\d+\.\d+)', re.S)
        k2 = re.compile(r'(\d+)', re.S)
        t = re.findall(k1, s) + re.findall(k2, s)
        t = float(t[0])
        if t == 0:
            raise IndexError
        return t

#检测time字符串是否符合日期格式
def check_time(time):
    try:
        datetime.datetime.strptime(time, '%Y-%m-%d')
    except ValueError:
        print(time)
        if time == '2015-06-31':
            time = '2015-07-01'
        if time[-2:] == 'NK':
            time = time[:-2]+'01'
    return time

#逐行读表并写入新表
def read_table(sheet, order, item, process):
    f = open(f'data/washed_sheet/{item}{order}.csv', 'w', encoding='gb2312', newline='')
    w = csv.writer(f)
    for i in range(1, sheet.nrows):
        row = sheet.row_values(i)
        t = process(row, order)
        if t[1] != '0000-00-00':
            w.writerow(t)

#合并同一个项目的两张表，并输出两张表的重合部分
def merge(task):
    overlap = 0 #两张表中重复的表项（同一个病人同一天两个记录）
    conflict = 0 #两张表中冲突的表项（同一个病人同一天两个记录不一样）
    result = {}
    f1 = open(f'data/washed_sheet/{task}1.csv', 'r')
    f2 = open(f'data/washed_sheet/{task}2.csv', 'r')
    sheet1 = csv.reader(f1)
    for row in sheet1:
        result[(int(row[0]), row[1])] = row[2:]
    sheet2 = csv.reader(f2)
    for row in sheet2:
        if (int(row[0]), row[1]) in result.keys():
            overlap += 1
            if row[2:] != result[(int(row[0]), row[1])]:
                conflict += 1
            if task != 'blood':
                result[(int(row[0]), row[1])] = row[2:]
        else:
            result[(int(row[0]), row[1])] = row[2:]
    print(f'ovrelap:{overlap}\tconflict:{conflict}')
    print(f'{len(result.keys())} records in total')
    #合并
    w = csv.writer(open(f'data/washed_sheet/{task}.csv','w',newline=''))
    for k,v in result.items():
        w.writerow([k[0],k[1]]+v)
    
    f1.close()
    f2.close()
    os.remove(f'data/washed_sheet/{task}1.csv')
    os.remove(f'data/washed_sheet/{task}2.csv')

#在冲突的数据中（同一个病人同一天有两条检测记录）选出一个
def choose_data(l1, l2):
    count1 = sum([1 for i in l1 if i != ''])
    count2 = sum([1 for i in l2 if i != ''])
    return l1 if count1 >= count2 else l2

#清洗两张表，并存储到{item}1/2.csv中。包括去重，补全，pro：具体的表项处理函数
def clean(sheet1, sheet2, item, pro):
    #去重
    index = {'blood': [[5, 6, 7],[7,5,6]], 'thyroid':[list(range(5,12)),list(range(5,12))],
            'ana18':[list(range(5,24)),list(range(5,24))],'aqp4igg':[[5,5],[5,6]],
            'vitd':[[6,10],[5,6]],'t_b':[list(range(5,14))+[19],list(range(5,14))+[19]],
            'ig3':[list(range(5,8)),list(range(5,8))],'il_6':[[5],[5]],
            'wbc':[[5],[5]],'csf':[[5],[7]],'ocb':[[5],[5]],'medicine':[[4],[4]]} #不同检测项目提取数据所在列
    result1 = {}
    result2 = {}
    for i in range(1, sheet1.nrows):
        row = sheet1.row_values(i)
        id = int(row[0])
        time = check_time(row[3])
        if time == '0000-00-00':#如果时间是0000-00-00舍弃这条数据
            continue
        #提取每一行的原始数据
        data = [row[j] for j in index[item][0]]
        if (id, time) in result1.keys():
            result1[(id, time)] = choose_data(result1[(id, time)], data)
            continue
        result1[(id,time)] = data
    for i in range(1, sheet2.nrows):
        row = sheet2.row_values(i)
        id = int(row[0])
        time = check_time(row[3])
        if time == '0000-00-00':
            continue
        data = [row[j] for j in index[item][1]]
        if (id, time) in result2.keys():
            result2[(id,time)] = choose_data(result2[(id,time)],data)
            continue
        result2[(id,time)] = data
    #存表
    f1 = open(f'data/washed_sheet/{item}1.csv','w', encoding='gb2312',newline='')
    f2 = open(f'data/washed_sheet/{item}2.csv','w', encoding='gb2312',newline='')
    w1 = csv.writer(f1)
    w2 = csv.writer(f2)

    num1 = num2 = 0
    for k,v in result1.items():
        valid, line = pro(v)
        if valid:
            num1 += 1
            w1.writerow([k[0],k[1]]+line)
    for k,v in result2.items():
        valid, line = pro(v)
        if valid:
            num2 += 1
            w2.writerow([k[0],k[1]]+line)
    print(f'{num1}/{num2} records left after duplicate removal')


### patients

In [21]:
print(f'Sheet 1\n\trows:{patients_1.nrows - 1}')
print(f'Sheet 2\n\trows:{patients_2.nrows - 1}')

Sheet 1
	rows:515
Sheet 2
	rows:933


In [22]:
def clean_patients(sheet, order):
    def get_diagnosis(s):
        res = [0, 0, 0, 0]
        s = s.upper()
        if s.__contains__("NMOSD"):
            res[0] = 1
        elif s.__contains__("NMO"):
            res[1] = 1
        if s.__contains__("PSS") or s.__contains__("干燥综合征"):
            res[2] = 1
        if s.__contains__("SLE"):
            res[3] = 1
        return res
    def pro(row, order):
        id = int(row[0])
        gender = 1 if row[1].__contains__('男') else 0
        birthday = check_time(row[2])
        diagnosis = get_diagnosis(row[8])
        return [id, birthday, gender] + diagnosis
    read_table(sheet, order, 'patients', pro)

clean_patients(patients_1, 1)
clean_patients(patients_2, 2)
merge('patients')


0000-00-00
ovrelap:515	conflict:0
932 records in total


### visit

In [23]:
print(f'Sheet 1\n\trows:{visit_1.nrows - 1}\n\tdeath:{sum([1 for i in visit_1.col_values(5) if i.__contains__("去世") or i.__contains__("死亡")])}')
print(f'Sheet 1\n\trows:{visit_2.nrows - 1}\n\tdeath:{sum([1 for i in visit_2.col_values(5) if i.__contains__("去世") or i.__contains__("死亡")])}')

Sheet 1
	rows:4200
	death:5
Sheet 1
	rows:5989
	death:6


In [24]:
def visit(sheet, order):
    def pro(row, order):
        id = int(row[0])
        time = check_time(row[4])
        return [id, time]
    read_table(sheet, order, 'visit', pro)

visit(visit_1, 1)
visit(visit_2, 2)
merge('visit')

0000-00-00
ovrelap:4200	conflict:0
5988 records in total


### breakout

In [3]:
print(f'Sheet 1\n\trows:{breakout_1.nrows - 1}')
print(f'Sheet 2\n\trows:{breakout_2.nrows - 1}')
print(f'\tsymptom censoring:{sum([1 for i in breakout_2.col_values(5)[1:] if i == "" or i.__contains__("其他")])}')
print(f'\toutcome censoring:{sum([1 for i in breakout_2.col_values(7)[1:] if i == "" or i.__contains__("不详")])}')

Sheet 1
	rows:2005
Sheet 2
	rows:2965
	symptom censoring:29
	outcome censoring:145


In [4]:
def breakout(sheet, order):
    #发作症状，没用上
    def get_symptom(s):
        res = [0, 0, 0, 0]
        # 基本上是这六个发病位置，有极少的其他和删失，都用默认值
        if s.__contains__("视神经"):
            res[0] = 1
        if s.__contains__('脊髓') or s.__contains__('尿/便障碍'):
            res[1] = 1
        if s.__contains__("大脑"):
            res[2] = 1
        if s.__contains__("小脑"):
            res[3] = 1
        return res
    #治疗方式，没用上
    def get_treatment(s):
        res = [0, 0, 0, 0]
        s = s.upper()
        if s.__contains__("激素"):
            res[0] = 1
        if s.__contains__("IVIG"):
            res[1] = 1
        if s.__contains__("中"):  #中医/中药
            res[2] = 1
        if s.__contains__("其他"):
            res[3] = 1
        return res
    #转归，没用上
    def get_outcome(s):
        if s.__contains__("完全"):
            return [1]
        if s.__contains__("无"):
            return [0]
        if s.__contains__('部分'):
            return [0.5]
        return [-1]
    def pro(row, order):
        id = int(row[0])
        time = check_time(row[4])
        symptom = get_symptom(row[5])
        treatment = get_treatment(row[5])
        outcome = get_outcome(row[7])
        return [id, time] + symptom + treatment + outcome
    read_table(sheet, order, 'breakout', pro)
breakout(breakout_1, 1)
breakout(breakout_2, 2)
merge('breakout')

2015-06-31
2017-05-NK
2015-06-31
2017-05-NK
ovrelap:2005	conflict:0
2965 records in total


### edss

#### sheet 1

In [27]:
names = edss_1.row_values(0)[14]
print(f'rows:\t{edss_1.nrows - 1}')
print(f'censoring:\n\t{names}:{sum([1 for i in edss_1.col_values(14)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, edss_1.nrows):
    row = edss_1.row_values(i)
    if (int(row[0]),row[4]) not in result.keys():
        result[(int(row[0]),row[4])] = []
    result[(int(row[0]), row[4])].append(row[14])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	2278
censoring:
	得分:0
conflict:


#### sheet 2

In [28]:
names = edss_2.row_values(0)[14]
print(f'rows:\t{edss_2.nrows - 1}')
print(f'censoring:\n\t{names}:{sum([1 for i in edss_2.col_values(14)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, edss_2.nrows):
    row = edss_2.row_values(i)
    if (int(row[0]),row[4]) not in result.keys():
        result[(int(row[0]),row[4])] = []
    result[(int(row[0]), row[4])].append(row[14])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	3002
censoring:
	得分:0
conflict:


In [29]:
def edss(sheet, order):
    def pro(row, order):
        id = int(row[0])
        time = check_time(row[4])
        edss = row[14]
        return [id, time, edss]
    read_table(sheet, order, 'edss', pro)

edss(edss_1, 1)
edss(edss_2, 2)
merge('edss')

ovrelap:2278	conflict:0
3002 records in total


### blood

#### sheet 1

In [5]:
names = blood_1.row_values(0)[5:8]
print(f'rows:\t{blood_1.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in blood_1.col_values(5+i)[1:] if j == ""])}')
print('conflict:')
result = {}
for i in range(1, blood_1.nrows):
    row = blood_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:8])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	2981
censoring:
	白细胞计数:5
	LY%:248
	NEUT%:256
conflict:
	(107083001, '2014-11-24'):	[[7.61, 22.7, 73.0], [7.61, '', '']]
	(101789021, '2015-12-22'):	[[11.32, 35.2, 58.8], [11.32, '', '']]
	(102931087, '2016-04-12'):	[[6.12, 27.9, 63.9], [6.12, '', '']]
	(107082985, '2014-11-20'):	[[12.81, 12.5, 84.6], [12.81, '', '']]
	(107083293, '2014-12-15'):	[[4.3, 37.9, 52.2], [4.3, '', '']]
	(101805533, '2015-10-27'):	[[5.49, '', ''], [5.49, '', '']]
	(101749963, '2015-08-15'):	[[5.4, '', ''], [5.4, '', '']]
	(107082653, '2015-04-14'):	[[8.54, 29.6, 66.2], [8.54, '', '']]
	(103667153, '2015-05-08'):	[[14.11, 13.1, 82.5], [14.11, '', '']]
	(101967351, '2016-02-29'):	[[7.02, 35.6, 57.9], [7.02, '', '']]
	(103693467, '2015-11-09'):	[[12.58, 24.1, 70.1], [12.58, '', '']]
	(101884943, '2015-06-16'):	[[7.75, 31.4, 62.6], [7.75, '', '']]
	(107082925, '2015-03-16'):	[[5.3, 18.6, 74.5], [5.3, '', '']]
	(103525345, '2016-05-16'):	[[8.94, '', ''], [8.94, '', '']]
	(107082945, '2014-12-30'):	[[10.11, 4.

#### sheet 2

In [6]:
names = blood_2.row_values(0)[5:8]
print(f'rows:\t{blood_2.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in blood_2.col_values(5+i)[1:] if j == ""])}')
print('conflict:')
result = {}
for i in range(1, blood_2.nrows):
    row = blood_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:8])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	3998
censoring:
	LY%:1315
	NEUT%:1327
	WBC:16
conflict:
	(107083001, '2014-11-24'):	[['', '', '13.10'], ['', '', '13.21']]
	(101789021, '2015-12-22'):	[['', '', '5.96'], [37.2, 54.8, '5.88']]
	(102931087, '2016-04-12'):	[['', '', '10.73'], [22.3, 69.7, '8.07']]
	(107082985, '2014-11-20'):	[['16.5 降低', '76.7 升高', '5.63'], ['19.9 降低', 73.8, '5.27']]
	(107083293, '2014-12-15'):	[['', '', '7.85'], ['', '', '1.27']]
	(101805533, '2015-10-27'):	[['19.9 降低', 73.2, '3.81'], ['17.1 降低', '78.2 升高', '4.92']]
	(101749963, '2015-08-15'):	[['', '', '11.55'], ['', '', '12.37']]
	(107082653, '2015-04-14'):	[['', '', '7.12'], [35.3, 57.6, '8.86']]
	(103667153, '2015-05-08'):	[['', '', '8.23'], ['', '', '6.52']]
	(101967351, '2016-02-29'):	[['', '', '4.82'], ['', '', '6.34']]
	(103693467, '2015-11-09'):	[[32.4, 54.6, '5.65'], ['14.9 降低', '80.9 升高', '14.69']]
	(101884943, '2015-06-16'):	[[34.2, 61.7, '6.64'], [25.6, 65.7, '6.48']]
	(107082925, '2015-03-16'):	[[28.5, 52.9, '1.93'], ['44.6 升高', 50.2,

In [7]:

#从原始表项获取血常规检测数据
def pro(list):
    default = default_data["blood_test"]
    max = [100, 100, 100]
    res = []
    for i in range(len(list)):
        try:
            t = str_to_float(list[i])
            if t < max[i]:
                res.append(t)
            else:
                raise IndexError
        except IndexError:
            # 数据缺失
            res.append(default[i])
    res[1] = res[0] * res[1] * 0.01
    res[2] = res[0] * res[2] * 0.01
    return True,res

#清洗
clean(blood_1,blood_2,'blood',pro)


#合并
merge('blood')

2964/3981 records left after duplicate removal
ovrelap:2964	conflict:2964
3981 records in total


In [11]:
names = blood_new.row_values(0)[7:13]
print(f'rows:\t{blood_new.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in blood_new.col_values(7+i)[1:] if j == ""])}')
print('conflict:')
result = {}
for i in range(1, blood_new.nrows):
    row = blood_new.row_values(i)
    if (int(row[0]),row[5]) not in result.keys():
        result[(int(row[0]),row[5])] = []
    result[(int(row[0]), row[5])].append(row[7:13])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	4451
censoring:
	白细胞计数（x10^9/L）:7
	NEUT#（x10^9/L）:829
	LY#（x10^9/L）:4077
	RBC（x10^12/L）:16
	HGB（g/L）:15
	PLT（x10^9/L）:28
conflict:
	(102972811, '2022-06-01'):	[['9.11', '5.721', '2.642', '4.56', '141', '336'], ['9.11', '5.721', '2.642', '4.56', '141', '336']]
	(102972627, '2022-02-25'):	[['9.07', '5.48', '3', '4.24', '135', '202'], ['9.07', '5.48', '3', '4.24', '135', '202']]
	(102931087, '2016-04-12'):	[['6.12', '', '', '3.95', '131', '252'], ['6.12', '', '', '3.95', '131', '252']]
	(104664335, '2015-05-14'):	[['9.72', '', '', '4.95', '146', '321.0|升高'], ['9.72', '', '', '4.95', '146', '321']]
	(101967351, '2016-02-29'):	[['7.02', '', '', '5.95|升高', '119', '208'], ['7.02', '', '', '5.95', '119', '208']]
	(101909619, '2016-01-25'):	[['5.94', '', '', '4.55', '133', '312'], ['5.94', '', '', '4.55', '133', '312']]
	(101884943, '2015-06-16'):	[['7.75', '', '', '4.45', '132', '177'], ['7.75', '', '', '4.45', '132', '177']]
	(103811719, '2022-01-21'):	[['6.23', '3.73', '1.97', '4.56', 

### thyroid

#### sheet 1

In [33]:
names = thyroid_1.row_values(0)[5:12]
print(f'rows:\t{thyroid_1.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in thyroid_1.col_values(5+i)[1:] if j == ""])}')
print('conflict:')
result = {}
for i in range(1, thyroid_1.nrows):
    row = thyroid_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:12])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	563
censoring:
	FT3:6
	FT4:6
	T3:16
	T4:17
	TSH3:8
	A-Tg:20
	A-TPO:20
conflict:
	(102051979, '2016-02-22'):	[[2.43, 1.378, 0.936, 9.51, 0.571, 37.06, 8.45], [2.43, 1.378, 0.936, 9.51, 0.571, 37.06, 8.45]]
	(103166519, '2014-01-09'):	[[3.04, 1.227, '', '', 0.651, '', ''], [3.04, 1.227, '', '', 0.651, '', '']]
	(102530541, '2014-11-18'):	[[3.47, 1.314, 1.214, 8.69, '4.684 升高', 96.51, '>600 升高'], [3.47, 1.314, 1.214, 8.69, 4.684, 96.51, '>600.0']]
	(102036229, '2016-03-08'):	[['4.11 升高', 1.271, '1.924 升高', 9.32, 2.772, 11.09, 9.9], [4.11, 1.271, 1.924, 9.32, 2.772, 11.09, 9.9]]
	(104664335, '2015-05-14'):	[[2.93, 1.34, 0.73, 7.5, 1.433, '158.7 升高', '<28.0'], [2.93, 1.34, 0.73, 7.5, 1.433, 158.7, '<28.0']]


#### sheet 2

In [34]:
names = thyroid_2.row_values(0)[5:12]
print(f'rows:\t{thyroid_2.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in thyroid_2.col_values(5+i)[1:] if j == ""])}')
print('conflict:')
result = {}
for i in range(1, thyroid_2.nrows):
    row = thyroid_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:12])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	937
censoring:
	FT3:8
	FT4:8
	T3:21
	T4:22
	TSH3:11
	A-Tg:28
	A-TPO:29
conflict:
	(102051979, '2016-02-22'):	[['2.43', '1.378', '0.936', '9.51', '0.571', '37.06', '8.45'], ['2.43', '1.378', '0.936', '9.51', '0.571', '37.06', '8.45']]
	(103166519, '2014-01-09'):	[['3.04', '1.227', '', '', '0.651', '', ''], ['3.04', '1.227', '', '', '0.651', '', '']]
	(102530541, '2014-11-18'):	[['3.47', '1.314', '1.214', '8.69', '4.684', '96.51', '>600'], ['3.47', '1.314', '1.214', '8.69', '4.684', '96.51', '>600.0']]
	(102036229, '2016-03-08'):	[['4.11', '1.271', '1.924', '9.32', '2.772', '11.09', '9.9'], ['4.11', '1.271', '1.924', '9.32', '2.772', '11.09', '9.9']]
	(104664335, '2015-05-14'):	[['2.93', '1.34', '0.73', '7.5', '1.433', '158.7', '<28.0'], ['2.93', '1.34', '0.73', '7.5', '1.433', '158.7', '<28.0']]


In [35]:
#获取甲状腺功能检测数据
def pro(list):
    # 填入16-17表格的平均值
    default = default_data["thyroid"]
    max = [10, 5, 3, 20, 15, 1500, 700]
    res = []
    for i in range(len(list)):
        try:
            t = str_to_float(list[i])
            if t < max[i] and t != 0:
                res.append(t)
            else:
                raise IndexError
        except IndexError:
            # 数据缺失
            res.append(default[i])
    return True,res

clean(thyroid_1,thyroid_2,'thyroid',pro)

merge('thyroid')


558/932 records left after duplicate removal
ovrelap:558	conflict:1
932 records in total


### ana18

#### sheet 1

In [36]:
names = ana18_1.row_values(0)[5:24]
print(f'rows:\t{ana18_1.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in ana18_1.col_values(5+i)[1:] if not (j.__contains__("阴") or j.__contains__("阳"))])}')
print('conflict:')
result = {}
for i in range(1, ana18_1.nrows):
    row = ana18_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:12])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	696
censoring:
	抗核抗体（IgG型）:4
	抗双链DNA抗体（IgG型）:1
	抗细胞浆抗体:9
	抗中心粒抗体:8
	抗Sm抗体（LIA）:7
	抗RNP抗体（LIA）:7
	抗SSA抗体（LIA）:6
	抗SSB抗体（LIA）:8
	抗Sc1-70抗体（LIA）:8
	抗Jo-1抗体（LIA）:9
	抗核糖体抗体（LIA）:10
	增值性核抗原抗体（LIA）:8
	抗组蛋白抗体（LIA）:9
	抗Ro 52抗体（LIA）:6
	抗PM-Scl抗体（LIA）:6
	抗核小体抗体（LIA）:6
	抗着丝点B抗体（LIA）:6
	抗线粒体抗体M2亚型（LIA）:6
	DNP乳胶凝集试验（LIA）:8
conflict:
	(101789021, '2015-12-22'):	[['', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(102051979, '2016-02-22'):	[['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(107082985, '2014-11-20'):	[['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(103362407, '2015-02-25'):	[['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(105068845, '2016-05-26'):	[['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阳性'], ['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阳性']]
	(102530541, '2014-11-18'):	[['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阳性', '阴性', '阴性'

#### sheet 2

In [37]:
names = ana18_2.row_values(0)[5:24]
print(f'rows:\t{ana18_2.nrows - 1}')
print('censoring:')     #统计缺失情况
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in ana18_2.col_values(5+i)[1:] if not (j.__contains__("阴") or j.__contains__("阳"))])}')
print('conflict:')
result = {}
for i in range(1, ana18_2.nrows):
    row = ana18_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:12])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	1106
censoring:
	抗核抗体（IgG型）:4
	抗双链DNA抗体（IgG型）:1
	抗细胞浆抗体:10
	抗中心粒抗体:9
	抗Sm抗体（LIA）:8
	抗RNP抗体（LIA）:8
	抗SSA抗体（LIA）:7
	抗SSB抗体（LIA）:9
	抗Sc1-70抗体（LIA）:9
	抗Jo-1抗体（LIA）:10
	抗核糖体抗体（LIA）:11
	增值性核抗原抗体（LIA）:9
	抗组蛋白抗体（LIA）:11
	抗Ro 52抗体（LIA）:7
	抗PM-Scl抗体（LIA）:7
	抗核小体抗体（LIA）:7
	抗着丝点B抗体（LIA）:7
	抗线粒体抗体M2亚型（LIA）:7
	DNP乳胶凝集试验（LIA）:9
conflict:
	(101789021, '2015-12-22'):	[['', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(102051979, '2016-02-22'):	[['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(107082985, '2014-11-20'):	[['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阴性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(103362407, '2015-02-25'):	[['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性']]
	(105068845, '2016-05-26'):	[['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阳性'], ['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阳性']]
	(102530541, '2014-11-18'):	[['阳性', '阴性', '阴性', '阴性', '阴性', '阴性', '阴性'], ['阳性', '阴性', 

In [38]:
def pro(list):
    res = []
    for i in range(len(list)):
        if list[i].__contains__('阳'):
            return True,[1]
    return True,[0]
clean(ana18_1,ana18_2,'ana18',pro)
merge('ana18')

0000-00-00
0000-00-00
0000-00-00
0000-00-00
688/1098 records left after duplicate removal
ovrelap:688	conflict:0
1098 records in total


### AQP4IgG

#### sheet 1

In [39]:
names = aqp4igg_1.row_values(0)[5]
print(f'rows:\t{aqp4igg_1.nrows - 1}')
print(f'censoring:\n\t{names}:{sum([1 for i in aqp4igg_1.col_values(5)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, aqp4igg_1.nrows):
    row = aqp4igg_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	2777
censoring:
	AQP4IgG:9
conflict:
	(102070737, '2016-01-14'):	['阳性', '']
	(107082985, '2014-11-21'):	['阴性', '阴性']
	(101909625, '2014-03-26'):	['阳性', '阳性']
	(102530541, '2014-11-18'):	['阳性', '阳性']
	(107082565, '2015-12-14'):	['阴性', '阴性']
	(102342521, '2015-04-23'):	['阳性', '阳性']
	(107082951, '2014-12-10'):	['阳性', '阳性']
	(101665973, '2015-06-11'):	['阳性', '阳性']
	(107082945, '2014-12-25'):	['阳性', '阳性']
	(104225313, '2014-11-25'):	['阴性', '阴性', '阴性']


#### sheet 2

In [40]:
names = aqp4igg_2.row_values(0)[5]
print(f'rows:\t{aqp4igg_2.nrows - 1}')
print(f'censoring:\n\t{names}:{sum([1 for i in aqp4igg_2.col_values(5)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, aqp4igg_2.nrows):
    row = aqp4igg_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	4178
censoring:
	AQP4IgG:45
conflict:
	(102070737, '2016-01-14'):	['阳性', '']
	(107082985, '2014-11-21'):	['阴性', '阴性']
	(101909625, '2014-03-26'):	['阳性', '阳性']
	(102530541, '2014-11-18'):	['阳性', '阳性']
	(107082565, '2015-12-14'):	['阴性', '阴性']
	(102342521, '2015-04-23'):	['阳性', '阳性']
	(107082951, '2014-12-10'):	['阳性', '阳性']
	(101665973, '2015-06-11'):	['阳性', '阳性']
	(107082945, '2014-12-25'):	['阳性', '阳性']
	(104225313, '2014-11-25'):	['阴性', '阴性', '阴性']


In [41]:
def pro(list):
    k = re.compile(r'(\d+:\d+)', re.S)
    t = re.findall(k, list[1])
    if len(t) > 0:
        t = t[0].split(":")
        t = t[1] + "/" + t[0]
        return True,[eval(t)]
    if list[0].__contains__("强"):
        return True,[100]
    elif list[0].__contains__("弱"):
        return True,[10]
    elif list[0].__contains__("阳"):
        return True,[32]
    else:
        return True,[0]
clean(aqp4igg_1,aqp4igg_2,'aqp4igg',pro)
merge('aqp4igg')

2766/4167 records left after duplicate removal
ovrelap:2766	conflict:0
4167 records in total


### VitD

#### sheet 1

In [42]:
names = [vitd_1.row_values(0)[6], vitd_1.row_values(0)[10]]
print(f'rows:\t{vitd_1.nrows - 1}')
print('censoring:')
print(f'\t{names[0]}:{sum([1 for i in vitd_1.col_values(6)[1:] if i == ""])}')
print(f'\t{names[1]}:{sum([1 for i in vitd_1.col_values(10)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, vitd_1.nrows):
    row = vitd_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append((row[6],row[10]))
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	1634
censoring:
	25-OH-VitD（总）:17
	1,25-OH-VitD3:299
conflict:


#### sheet 2

In [43]:
names = [vitd_2.row_values(0)[5], vitd_2.row_values(0)[6]]
print(f'rows:\t{vitd_2.nrows - 1}')
print('censoring:')
print(f'\t{names[0]}:{sum([1 for i in vitd_2.col_values(5)[1:] if i == ""])}')
print(f'\t{names[1]}:{sum([1 for i in vitd_2.col_values(6)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, vitd_2.nrows):
    row = vitd_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append((row[5],row[6]))
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	803
censoring:
	25-OH-VitD（总）:5
	1,25-OH-VitD3:18
conflict:


In [44]:
def pro(list):
    default = default_data['vitd']
    try:
        vitd = str_to_float(list[0])
    except IndexError:
        vitd = default[0]
    try:
        vitd2 = str_to_float(list[1])
    except IndexError:
        vitd2 = default[1]
    return True,[vitd,vitd2]
clean(vitd_1,vitd_2,'vitd',pro)
merge('vitd')

1634/803 records left after duplicate removal
ovrelap:0	conflict:0
2437 records in total


### T-B

#### sheet 1

In [45]:
names = t_b_1.row_values(0)[5:14] + [t_b_1.row_values(0)[19]]
print(f'rows:\t{t_b_1.nrows - 1}')
print('censoring:')
for i in range(len(names) - 1):
    print(f'\t{names[i]}:{sum([1 for j in t_b_1.col_values(5+i)[1:] if j == ""])}')
print(f'\t{names[-1]}:{sum([1 for i in t_b_1.col_values(19)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, t_b_1.nrows):
    row = t_b_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:14]+[row[19]])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	1444
censoring:
	B淋巴细胞CD19＋:3
	B淋巴细胞CD19＋ (%):0
	自然杀伤细胞CD16＋CD56＋:3
	自然杀伤细胞CD16＋CD56＋ (%):3
	T淋巴细胞CD3＋:4
	T淋巴细胞CD3＋ (%):1
	T辅助细胞 CD3＋/CD4＋:10
	T抑制/细胞毒细胞CD3＋/CD8＋:10
	T抑制/细胞毒细胞CD3＋/CD8＋ (%):27
	T淋巴细胞CD4＋:142
conflict:


#### sheet 2

In [46]:
names = t_b_2.row_values(0)[5:14] + [t_b_2.row_values(0)[19]]
print(f'rows:\t{t_b_2.nrows - 1}')
print('censoring:')
for i in range(len(names) - 1):
    print(f'\t{names[i]}:{sum([1 for j in t_b_2.col_values(5+i)[1:] if j == ""])}')
print(f'\t{names[-1]}:{sum([1 for i in t_b_2.col_values(19)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, t_b_2.nrows):
    row = t_b_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:14]+[row[19]])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	995
censoring:
	B淋巴细胞CD19＋:2
	B淋巴细胞CD19＋ (%):1
	自然杀伤细胞CD16＋CD56＋:2
	自然杀伤细胞CD16＋CD56＋ (%):2
	T淋巴细胞CD3＋:2
	T淋巴细胞CD3＋ (%):2
	T辅助细胞 CD3＋/CD4＋:1
	T抑制/细胞毒细胞CD3＋/CD8＋:1
	T抑制/细胞毒细胞CD3＋/CD8＋ (%):6
	T淋巴细胞CD4＋:41
conflict:


In [47]:
def pro(list):
    
    # 填入16-17表格的平均值
    default = default_data["t_b"]
    max = [2500, 100, 2000, 100, 7000, 100, 4000, 3000, 100, 100]
    res = []
    for i in range(len(list)):
        try:
            t = str_to_float(list[i])
            if t < max[i]:
                res.append(t)
            else:
                raise IndexError
        except IndexError:
            # 数据缺失
            res.append(default[i])
    return True,res
clean(t_b_1,t_b_2,'t_b',pro)
merge('t_b')

1444/995 records left after duplicate removal
ovrelap:0	conflict:0
2439 records in total


### IG3

#### sheet 1

In [48]:
names = ig3_1.row_values(0)[5:8]
print(f'rows:\t{ig3_1.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in ig3_1.col_values(5+i)[1:] if j == ""])}')
print('conflict:')
result = {}
for i in range(1, ig3_1.nrows):
    row = ig3_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:8])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	1381
censoring:
	IgG:1
	IgA:1
	IgM:1
conflict:


#### sheet 2

In [49]:
names = ig3_2.row_values(0)[5:8]
print(f'rows:\t{ig3_2.nrows - 1}')
print('censoring:')
for i in range(len(names)):
    print(f'\t{names[i]}:{sum([1 for j in ig3_2.col_values(5+i)[1:] if j == ""])}')
print('conflict:')
result = {}
for i in range(1, ig3_2.nrows):
    row = ig3_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5:8])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	960
censoring:
	IgG:4
	IgA:4
	IgM:5
conflict:


In [50]:
def pro(list):
    default = default_data['ig3']
    try:
        igg = str_to_float(list[0])
    except IndexError:
        igg = default[0]
    try:
        iga = str_to_float(list[1])
        if iga > 10:
            raise IndexError
    except IndexError:
        iga = default[1]
    try:
        igm = str_to_float(list[2])
    except IndexError:
        igm = default[2]
    return True,[igg,iga,igm]
clean(ig3_1,ig3_2,'ig3',pro)
merge('ig3')

1381/960 records left after duplicate removal
ovrelap:0	conflict:0
2341 records in total


### IL-6

#### sheet 1

In [51]:
names = il_6_1.row_values(0)[5]
print(f'rows:\t{il_6_1.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in il_6_1.col_values(5)[1:] if i == ""])}')
print('conflict:')
result = {}
for i in range(1, il_6_1.nrows):
    row = il_6_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	1299
censoring:
	IL-6:4
conflict:


#### sheet 2

In [52]:
names = il_6_2.row_values(0)[5]
print(f'rows:\t{il_6_2.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in il_6_2.col_values(5)[1:] if i == ""])}')
print(f'conflict:')
result = {}
for i in range(1, il_6_2.nrows):
    row = il_6_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	946
censoring:
	IL-6:0
conflict:


In [53]:
def pro(list):
    try:
        t = str_to_float(list[0])
        if t < 30:
            return True,[t]
        else:
            raise IndexError
    except IndexError:
        return True,default_data['il_6']

clean(il_6_1,il_6_2,'il_6',pro)
merge('il_6')

1299/946 records left after duplicate removal
ovrelap:0	conflict:0
2245 records in total


### 脑脊液常规WBC

#### sheet 1

In [54]:
names = wbc_1.row_values(0)[5]
print(f'rows:\t{wbc_1.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in wbc_1.col_values(5)[1:] if i == ""])}')
print(f'conflict:')
result = {}
for i in range(1, wbc_1.nrows):
    row = wbc_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	375
censoring:
	WBC细胞计数:67
conflict:
	(102051979, '2016-02-23'):	[2.0, 2.0]
	(105350297, '2016-07-29'):	[2.0, 2.0]
	(101665973, '2015-06-11'):	[6.0, 6.0]


#### sheet 2

In [55]:
names = wbc_2.row_values(0)[5]
print(f'rows:\t{wbc_2.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in wbc_2.col_values(5)[1:] if i == ""])}')
print(f'conflict:')
result = {}
for i in range(1, wbc_2.nrows):
    row = wbc_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	211
censoring:
	WBC细胞计数:2
conflict:


In [56]:
def pro(list):
    try:
        t = str_to_float(list[0])
        return True,[t]
    except:
        return True,default_data['wbc']
clean(wbc_1,wbc_2,'wbc',pro)
merge('wbc')

0000-00-00
0000-00-00
370/211 records left after duplicate removal
ovrelap:0	conflict:0
581 records in total


### CSF(脑脊液生化)

#### sheet 1

In [57]:
names = csf_1.row_values(0)[5]
print(f'rows:\t{csf_1.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in csf_1.col_values(5)[1:] if i == ""])}')
print(f'conflict:')
result = {}
for i in range(1, csf_1.nrows):
    row = csf_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	307
censoring:
	蛋白:0
conflict:
	(102051979, '2016-02-23'):	[0.3, 0.3]
	(105350297, '2016-07-29'):	['0.47 升高', 0.47]
	(101665973, '2015-06-11'):	[0.35, 0.35]


#### sheet 2

In [58]:
names = csf_2.row_values(0)[7]
print(f'rows:\t{csf_2.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in csf_2.col_values(7)[1:] if i == ""])}')
print(f'conflict:')
result = {}
for i in range(1, csf_2.nrows):
    row = csf_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[7])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	211
censoring:
	蛋白:0
conflict:


In [59]:
def pro(list):
    try:
        t = str_to_float(list[0])
        if t < 10:
            return True,[t]
        else:
            raise IndexError
    except IndexError:
        return True,[0.4]
clean(csf_1,csf_2,'csf',pro)
merge('csf')

304/211 records left after duplicate removal
ovrelap:0	conflict:0
515 records in total


### OCB

#### sheet 1

In [60]:
names = ocb_1.row_values(0)[5]
print(f'rows:\t{ocb_1.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in ocb_1.col_values(5)[1:] if i == "" or i == "未查"])}')
print(f'conflict:')
result = {}
for i in range(1, ocb_1.nrows):
    row = ocb_1.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	400
censoring:
	OCB(寡克隆带）:30
conflict:
	(102051979, '2016-02-23'):	['1型：血清及脑脊液均无条带', '1型：血清及脑脊液均无条带']
	(103699295, '2015-06-11'):	['4型：血清与脑脊液相同条带 呈镜像分布', '4型：血清与脑脊液相同条带 呈镜像分布']
	(105350297, '2016-07-29'):	['未查', '未查']
	(101665973, '2015-06-11'):	['1型：血清及脑脊液均无条带', '1型：血清及脑脊液均无条带']


#### sheet 2

In [61]:
names = ocb_2.row_values(0)[5]
print(f'rows:\t{ocb_2.nrows - 1}')
print('censoring:')
print(f'\t{names}:{sum([1 for i in ocb_2.col_values(5)[1:] if i == "" or i == "未查"])}')
print(f'conflict:')
result = {}
for i in range(1, ocb_2.nrows):
    row = ocb_2.row_values(i)
    if (int(row[0]),row[3]) not in result.keys():
        result[(int(row[0]),row[3])] = []
    result[(int(row[0]), row[3])].append(row[5])
for k,v in result.items():
    if len(v) > 1:
        print(f'\t{k}:\t{v}')

rows:	243
censoring:
	OCB(寡克隆带）:0
conflict:


In [62]:
def pro(list):
    k = re.compile(r'(\d)', re.S)
    try:
        n = int(re.findall(k,list[0])[0])
    except IndexError:
        n = -1
    if n != -1:
        if n == 1 or n == 4:
            return True,[0]
        else:
            return True,[1]
    return False,[-1]
clean(ocb_1,ocb_2,'ocb',pro)
merge('ocb')

0000-00-00
0000-00-00
365/243 records left after duplicate removal
ovrelap:0	conflict:0
608 records in total


### medicine

In [63]:
print(f'sheet 1\n\trows:\t{medicine_1.nrows}\nsheet 2\n\trows:\t{medicine_2.nrows}')

sheet 1
	rows:	1549
sheet 2
	rows:	533


In [64]:
#用药表比较特殊，提取时间是用药开始的时间
def medicine(sheet, order):
    def get_medicine(str):
        res = [0, 0, 0, 0, 0, 0]
        if str.__contains__("骁悉") or str.__contains__("赛可平"):
            res[0] = 1
        if str.__contains__("硫唑嘌呤"):
            res[1] = 1
        if str.__contains__("纷乐"):
            res[2] = 1
        if str.__contains__("环磷酰胺"):
            res[3] = 1
        if str.__contains__("美罗华"):
            res[4] = 1
        if res == [0, 0, 0, 0, 0, 0]:
            res[5] = 1
        return res

    def pro(value, task):
        id = int(value[0])
        time = check_time(value[5])
        medicine = get_medicine(value[4])
        return [id, time] + medicine

    read_table(sheet, order, "medicine", pro)

medicine(medicine_1, 1)
medicine(medicine_2, 2)
merge('medicine')

2015-03-NK
ovrelap:532	conflict:16
1524 records in total
